In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import collections 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path  = "smaller_dada.h5ad"
adata = sc.read_h5ad(path)

In [3]:
def create_connectivity_matrix(adata):
    positions_in_tissue = adata.obs[adata.obs.columns[:3]][adata.obs.in_tissue ==1]
    barcodes_in_tissue = positions_in_tissue.index
    nbarcodes_in_tissue = len(positions_in_tissue)
    positions_in_tissue = positions_in_tissue.reset_index().rename(columns={'index':'_id'})

    C = np.zeros([nbarcodes_in_tissue, nbarcodes_in_tissue])
    for idx, barcode in enumerate(barcodes_in_tissue):

        row_i  = int(positions_in_tissue[positions_in_tissue['_id'] == barcode ]['array_row'])
        col_i= int(positions_in_tissue[positions_in_tissue['_id'] == barcode ]['array_col'])

        condition = ((positions_in_tissue['array_row'] == row_i-1 )&(positions_in_tissue['array_col'] == col_i-1))\
                    | ((positions_in_tissue['array_row'] == row_i-1 )&(positions_in_tissue['array_col'] == col_i+1))\
                    | ((positions_in_tissue['array_row'] == row_i )&(positions_in_tissue['array_col'] == col_i-2))\
                    | ((positions_in_tissue['array_row'] == row_i)&(positions_in_tissue['array_col'] == col_i+2))\
                    | ((positions_in_tissue['array_row'] == row_i+1 )&(positions_in_tissue['array_col'] == col_i-1))\
                    | (positions_in_tissue['array_row'] == row_i+1 )&(positions_in_tissue['array_col'] == col_i+1)
        tmp = positions_in_tissue[condition]

        if len(tmp) > 0:
            for j in tmp.index:
                C[idx, j] = 1

    row_sums = C.sum(1)
    row_sums[row_sums == 0] = 1e-14
    W = C / row_sums.reshape(-1, 1)

    conn_info = dict()
    conn_info['L_estimate_divR'] = np.diagonal((np.dot(W, W).T)).sum() / (nbarcodes_in_tissue - 1)
    conn_info['barcodes_in_tissue'] = barcodes_in_tissue.tolist()
    conn_info['nbarcodes_in_tissue'] = nbarcodes_in_tissue
    conn_info['W'] = W
    conn_info['C'] = C

    return conn_info

In [4]:
conn_info = create_connectivity_matrix(adata)

In [6]:
def calculate_bsc(feature1, feature2, adata, conn_info):
    gene_names = adata.var.index.tolist()
    row_col = adata.obs[['array_row', 'array_col']].values.astype(int)
    df = pd.DataFrame(data=np.concatenate((row_col, adata.X), axis=1), columns=['row', 'col'] + gene_names)

    x_values = df[feature1].values
    y_values = df[feature2].values

    x_mean = np.mean(x_values)
    y_mean = np.mean(y_values)

    x_smooth = np.dot(conn_info['W'], x_values)
    y_smooth = np.dot(conn_info['W'], y_values)

    x_mean_sm = np.mean(x_smooth) # muX
    y_mean_sm = np.mean(y_smooth) # muY

    # Calculate Peason's r(X,Y), r(smooth), L_XX, L_YY, L_XY as in Lee S (2001)
    r = sum((x_values - x_mean) * (y_values - y_mean)) \
       / (np.sqrt(sum((x_values - x_mean) ** 2)) * np.sqrt(sum((y_values - y_mean) ** 2)))
    r_sm = sum((x_smooth - x_mean_sm) * (y_smooth - y_mean_sm)) \
          / (np.sqrt(sum((x_smooth - x_mean_sm) ** 2)) * np.sqrt(sum((y_smooth - y_mean_sm) ** 2)))

    L_XX = sum((x_smooth - x_mean) ** 2) / sum((x_values - x_mean) ** 2)
    L_YY = sum((y_smooth - y_mean) ** 2) / sum((y_values - y_mean) ** 2)
    L_XY = np.sqrt(L_XX) * np.sqrt(L_YY) * r_sm

    bsc = {
        'r': r,
        'r_sm': r_sm,
        'L_XX': L_XX,
        'L_YY': L_YY,
        'L_XY': L_XY
    }

    return bsc

In [7]:
bsc = calculate_bsc('Ttr', 'Ecrg4', adata, conn_info)

In [8]:
bsc

{'r': 0.6207497681119728,
 'r_sm': 0.7803523788309347,
 'L_XX': 0.8702965163074358,
 'L_YY': 0.63744614853193,
 'L_XY': 0.5812274701776152}

In [16]:
import itertools
gene_names = adata.var.index.tolist()

for gene1, gene2 in itertools.combinations(gene_names, 2):
    print(gene1, gene2)